# Sad Poems

In [1]:
import numpy as np
import pandas as pd 
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
import requests as rq

Using TensorFlow backend.


In [2]:
# loading additional nature poems from poetryfoundation.com dataset
df2 = pd.read_csv('PoetryFoundationData.csv') # load dataset
df2 = df2.dropna()
df2.head() # getting top 5 head

,Unnamed: 0,Title,Poem,Poet,Tags
6,6,\r\r\n Invisible Fish\r\r\n...,\r\r\nInvisible fish swim this ghost ocean now...,Joy Harjo,"Living,Time & Brevity,Relationships,Family & A..."
7,7,\r\r\n Don’t Bother the Ear...,\r\r\nDon’t bother the earth spirit who lives ...,Joy Harjo,"Religion,The Spiritual,Mythology & Folklore,Fa..."
9,9,"\r\r\n [""Hour in which I co...","\r\r\nHour in which I consider hydrangea, a sa...",Simone White,"Living,Parenthood,The Body,The Mind,Nature,Tre..."
16,16,\r\r\n scars\r\r\n ...,\r\r\nmy father’s body is a map\r\r\na record ...,Truong Tran,"The Body,Family & Ancestors"
17,17,\r\r\n what remains two\r\r...,\r\r\nit has long been forgotten this practice...,Truong Tran,"Infancy,Parenthood,The Body"


In [3]:
searchfor = ['Sorrow']

In [4]:
# getting additional poems that contain the tag Love 
poems2 = df2['Poem']
sadPoems = poems2.loc[df2['Tags'].str.contains('|'.join(searchfor))]

In [5]:
sadPoems.head()


28    \r\r\nNothing can reach you now, not lead or s...
29    \r\r\nAmong physicians rich in their death wat...
30    \r\r\nImagine half your face\r\r\nrubbed out y...
47    \r\r\nThe   first    great    poet  of\r\r\nth...
73    \r\r\nWhen I was three,\r\r\na tsunami hit tow...
Name: Poem, dtype: object

In [6]:
#convert nature series to a string
sadPoems = sadPoems.to_string()

In [7]:
# MAPPING CHARACTERS: for nature poems, Make two dictionaries , one to convert chars to ints, the other to convert ints back to chars

chars = sorted(list(set(sadPoems)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
# process the dataset:
seqlen = 50
step = seqlen

data_X = []
data_y = []

poemLines = [] 

# creates poem lines 
for i in range(0, len(sadPoems) - seqlen - 1, step):
    poemLines.append(sadPoems[i: i + seqlen + 1])
    
# creating x and y data
data_X = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)
data_Y = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)

for i, poemLines in enumerate(poemLines):
    for t, (char_in, char_out) in enumerate(zip(poemLines[:-1], poemLines[1:])):
        data_X[i, t, char_indices[char_in]] = 1
        data_Y[i, t, char_indices[char_out]] = 1

In [9]:
# create the model
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['categorical_crossentropy', 'accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 128)           115712    
_________________________________________________________________
dense_1 (Dense)              (None, 50, 97)            12513     
Total params: 128,225
Trainable params: 128,225
Non-trainable params: 0
_________________________________________________________________


In [10]:
# train the model
model.fit(data_X, data_Y, batch_size=128, epochs=80)

Epoch 1/80
993/993 [==============================] - 2s 2ms/step - loss: 3.6518 - categorical_crossentropy: 3.6518 - accuracy: 0.1667
Epoch 2/80
993/993 [==============================] - 2s 2ms/step - loss: 3.1527 - categorical_crossentropy: 3.1527 - accuracy: 0.2185
Epoch 3/80
993/993 [==============================] - 2s 2ms/step - loss: 3.0622 - categorical_crossentropy: 3.0622 - accuracy: 0.2714
Epoch 4/80
993/993 [==============================] - 2s 2ms/step - loss: 2.9127 - categorical_crossentropy: 2.9127 - accuracy: 0.2976
Epoch 5/80
993/993 [==============================] - 2s 2ms/step - loss: 2.6151 - categorical_crossentropy: 2.6151 - accuracy: 0.3578
Epoch 6/80
993/993 [==============================] - 2s 2ms/step - loss: 2.3471 - categorical_crossentropy: 2.3471 - accuracy: 0.3997
Epoch 7/80
993/993 [==============================] - 2s 2ms/step - loss: 2.1913 - categorical_crossentropy: 2.1913 - accuracy: 0.4273
Epoch 8/80
993/993 [==============================] - 2

In [11]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)  

In [12]:
def generating_poem(epoch, start_index, diversity, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = start_index
    diversity = diversity

    generated = ''
    poemLines = sadPoems[start_index: start_index + seqlen]
    generated += poemLines
    print('----- Generating with seed: "' + poemLines + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(poemLines):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        poemLines = poemLines[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [13]:
generating_poem(80, 100, .25, _)


----- Generating text after Epoch: 80
----- Generating with seed: "heir death wat...
30       \r\r\nImagine half your"
heir death wat...
30       \r\r\nImagine half your back again, a caplow my dad...
6965     \r\r\nI saw afle the day haig the poated sain who toull...
13044    \r\r\nThe casken an the day is a which here a mome the ...
4319     \r\r\n\r\r\n\r\r\n\r\r\nSong ther and and o...
626      \r\r\nI saw must al my so ding the mouthers. I was not...
11888    \r\r\nI had a lotele has lood, to like there is go...
10706    \r\r\nI had my father said of the car
